In [ ]:
### MACHINE LEARNING 

import joblib
import numpy as np

def predict(coordinates):
    """
    By taking the coordinates as inputs, predict the clustering class from:
        1. 'not_crowded' : between 0-3 people
        2. 'usual': between 3-21 people
        3. 'crowded': between 21-109 people
        4. 'very_crowded': more than 109 people
    Input: [[lat, lon]]
    """
    # Load the models from disk
    # kmeans = joblib.load('C:/Users/Anik Barua/Downloads/HSL_ML/kmeans.pkl')
    kmeans = joblib.load('kmeans.pkl')
    model = joblib.load('svc.pkl')

    # Predict the clustering id given coordinates
    cluster = kmeans.predict(coordinates)

    # Predict the crowdness given the clustering id
    crowdness = model.predict([cluster])

    # Return crowdness
    return crowdness[0]
##############################################################################

def navigateRoute(origin, dest):
    from selenium import webdriver
    from selenium.webdriver.common.keys import Keys
    import time
    driver = webdriver.Firefox()

    base_address = "https://hsl.fi/en"
    driver.get(base_address)

    origin = driver.find_element_by_id("edit-from-in").send_keys(origin)
    time.sleep(0)
    dest = driver.find_element_by_id("edit-to-in").send_keys(dest)
    time.sleep(0)
    submit = driver.find_element_by_id("edit-submit").click()
    time.sleep(2)

import requests
import geopy
import time
import gmaps

from geopy.geocoders import Nominatim

def location_converter(location):
    geolocator = Nominatim()
    location = geolocator.geocode(location)
    return (location.latitude, location.longitude)

def run_query(query, api_link): 
    request = requests.post(api_link, json={'query': query})
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))
        

def compute_co2(origin, destination):
    #origin = "Vantaa"
    #destination = "Espoo"
    
    duration_minute = 0
    distance_km = 0
    carbon = 0
    
    origin_coordinates = location_converter(origin)
    
    ##########################
    
    coordinate_from = [[origin_coordinates[0], origin_coordinates[1]]]
    print("CO-ORDINATE: ",coordinate_from)
    status = predict(coordinate_from)
    print(status)
    #if status == 'crowded':
    #    print("Crowded")
    #elif status == 'very_crowded':
    #    print("Very Crowded")
    #############################
    
    print("Origin: ", origin_coordinates)
    destination_coordinates = location_converter(destination)
    print("Destination: ", destination_coordinates)
    
    #################################
    query = """
{
  
  plan(
    from: {lat: """+ str(origin_coordinates[0])+""", lon: """+str(origin_coordinates[1])+"""}
    to: {lat: """+ str(destination_coordinates[0])+""", lon: """+ str(destination_coordinates[1])+"""}
    numItineraries: 1
  ) {
    itineraries {
      legs {
        mode
        duration
        distance
      }}}}
"""

    api_link = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"

    ##################################
    



# Execute the query and extract the result

    result = run_query(query,api_link) 
    
    time.sleep(5)
    
    result_dictionary = result["data"]["plan"]["itineraries"]

    
    route_num = 0
    for legs in result_dictionary:
        route_num += 1
        dict_split = dict(legs)
        dict_route = dict_split['legs']
        # print("Route Number: {} \n{}".format(route_num, dict_route))
        print("\n")

        duration = 0 
        distance = 0 
        carbon = 0

        for route in dict_route:
            print(route)

            #Calculating duration in min 
            duration += route['duration']
            duration_minute = int(duration/60)

            #Calculating distance in km 
            distance += route['distance']
            distance_km = round(float(distance/1000),2)

            carbon_factor = 0.065
            if route['mode'] in ("RAIL","TRAM","METRO"):
                carbon_factor = 0.065
            if route['mode'] == "BUS":
                carbon_factor = 0.069
            if route['mode'] in ("WALK"):
                carbon_factor = 0
            carbon += round(float((carbon_factor * distance_km)),7)

        print("Total duration: {} min".format(duration_minute))
        print("Total distance: {} km".format(distance_km))
        print("Carbon emission: {} kg".format(carbon))
    return((duration_minute, distance_km, carbon, status))


#############################SERVER########################################
import flask
from flask import Flask, redirect, url_for, request
from flask import render_template

app = Flask(__name__, template_folder='templates')

fromData = ""
toData = ""

def gatherResult(a, b):
    print("got the data from: ", a, " and ", b)

@app.route('/success/<origin>?<dest>')
def success(origin, dest):
    fromData = origin
    toData = dest
    resultObject = compute_co2(origin, dest)
    distance = resultObject[0]
    duration = resultObject[1]
    co2 = resultObject[2]
    status = resultObject[3]
    mode = "public transport"
    data = {'transportMode': mode,'distance': distance, 'duration': duration, 'Co2' : co2, 'status' : status}
    gatherResult(fromData, toData)
    navigateRoute(origin, dest)
    time.sleep(2)
    return render_template('ttt.html', data=data)

@app.route('/login',methods = ['POST', 'GET'])
def login():
    if request.method == 'POST':
        fromAdd = request.form['origin']
        destAdd = request.form['dest']
        return redirect(url_for('success',origin = fromAdd, dest = destAdd))
    else:
        user = request.args.get('origin')
        user = request.args.get('dest')
        return redirect(url_for('success',origin = fromAdd, dest = destAdd))

if __name__ == '__main__':
    app.run(debug = False,host='localhost')
    

%tb

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Nov/2019 08:56:06] "POST /login HTTP/1.1" 302 -
/home/jamie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


CO-ORDINATE:  [[60.1674086, 24.9425683]]


/home/jamie/miniconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator KMeans from version 0.21.3 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jamie/miniconda3/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator SVC from version 0.21.3 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/jamie/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:54: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agen

not_crowded
Origin:  (60.1674086, 24.9425683)
Destination:  (60.2051454, 24.6569676)


127.0.0.1 - - [17/Nov/2019 08:56:13] "GET /success/Helsinki%3FEspoo HTTP/1.1" 200 -




{'mode': 'WALK', 'duration': 547.0, 'distance': 649.2400000000001}
{'mode': 'RAIL', 'duration': 1500.0, 'distance': 19929.455141700928}
{'mode': 'WALK', 'duration': 57.0, 'distance': 63.494}
Total duration: 35 min
Total distance: 20.64 km
Carbon emission: 1.3377 kg
got the data from:  Helsinki  and  Espoo
